# pySpATS Quick Start: Model Fitting and Diagnostics

This notebook demonstrates the core functionality of pySpATS:
- Fitting a spatial model to field trial data
- Viewing effective dimension (ED) summary
- Computing heritability
- Visualizing the fitted spatial surface

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspats import SpATS
from pyspats.datasets import load_dataset

%matplotlib inline

## Load Example Data

We'll use the wheat dataset that comes with pySpATS.

In [ ]:
# Load wheat field trial data
data = load_dataset('wheat')

print(f"Dataset shape: {data.shape}")
print(f"\nFirst few rows:")
print(data.head())

print(f"\nColumns: {list(data.columns)}")
print(f"Number of genotypes: {data['geno'].nunique()}")
print(f"Number of observations: {len(data)}")

## Fit SpATS Model

We fit a spatial model with:
- Response: yield
- Genotypes as fixed effects (for BLUE extraction)
- 2D spatial smooth using row and column coordinates
- Replication as random effect

In [ ]:
# Fit SpATS model
model = SpATS(
    response='yield',
    genotype='geno',
    spatial=('col', 'row'),
    random=['rep'],
    data=data,
    genotype_as_random=False  # Treat genotypes as fixed for BLUE extraction
)

print("Model fitted successfully!")
print(f"Deviance: {model.deviance:.2f}")
print(f"Iterations: {model.n_iterations}")

## Effective Dimension Summary

The `summary_ed()` method shows how degrees of freedom are allocated across model components:
- Fixed effects (intercept + genotypes)
- Spatial smooths (row, column, interaction)
- Random effects (replication)
- Residual

In [ ]:
# Display effective dimension breakdown
model.summary_ed()

## Heritability Estimation

PySpATS computes generalized heritability by default:
- H² = ED_geno / n_geno (SpATS-style)

For classical heritability: H² = ED_geno / (n_geno - 1), use `get_heritability(mode='classical')`

In [ ]:
# Generalized heritability (default)
h2_gen = model.heritability
print(f"Generalized Heritability: {h2_gen:.3f}")

# Classical heritability (for comparison)
h2_class = model.get_heritability(mode='classical')
print(f"Classical Heritability:   {h2_class:.3f}")

# Extract genotype BLUEs
blues = model.get_BLUEs()
print(f"\nExtracted BLUEs for {len(blues)} genotypes")
print("\nTop 5 genotypes:")
print(blues.sort_values('BLUE', ascending=False).head())

## Visualize Spatial Surface

The spatial smooth captures field heterogeneity patterns.

In [ ]:
# Plot spatial trend
model.plot_spatial()
plt.suptitle('Fitted Spatial Surface (Field Heterogeneity)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Full Diagnostic Plot

The 6-panel diagnostic provides comprehensive model assessment:
1. Spatial residuals
2. Fitted vs observed
3. Residual distribution
4. Genotype BLUEs distribution
5. Top/bottom performers
6. Variance components

In [ ]:
# Comprehensive diagnostic plot
model.plot(all_in_one=True, figsize=(15, 10))
plt.show()

## Variance Components

View the estimated variance for each model component.

In [ ]:
# Display variance components
print("Variance Components:")
print("-" * 40)
for component, var in model.var_comp.items():
    print(f"{component:30s}: {var:.6f}")
print(f"{'Residual (psi)':30s}: {model.psi:.6f}")

## Summary

This example demonstrated:
- ✅ Simple model fitting with spatial correction
- ✅ Effective dimension breakdown via `summary_ed()`
- ✅ Heritability estimation (generalized and classical)
- ✅ Spatial surface visualization
- ✅ Comprehensive diagnostics

For more advanced usage, see the main documentation and `pyspats_sorghum_example.py`.